In [198]:
import pandas as pd
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import VarianceThreshold
import numpy as np

In [2]:
df = pd.read_csv("offen.csv")

In [3]:
inputs = df.iloc[:,3:].values
outs = df["labels"].values



selector = SelectKBest(f_classif, k=5)
X_new = selector.fit_transform(inputs, outs)
cols = selector.get_support(indices=True)
selected_patterns = np.take(df.columns.values,[x+3 for x in cols] )
print(selected_patterns)

['[bitch]' '[bitch]|[hoe]' '[bitch]|$LOC' '[hoe]|[bitch]' '$DATE|[bitch]']


In [4]:

df_new = df.select_dtypes(include='number').drop("id", axis = 1)

In [5]:
matrix = df_new.corr().abs()

In [6]:
mask = np.triu(np.ones_like(matrix, dtype=bool))

reduced_matrix = matrix.mask(mask)

In [7]:
reduced_matrix.iloc[:5,:5]

,labels,NUM,NUM|[bitch],NUM|[bitch]+NOUN,NUM|[bitch]+NOUN|AUX
labels,NaN,NaN,NaN,NaN,NaN
NUM,0.117709,NaN,NaN,NaN,NaN
NUM|[bitch],0.445058,0.745356,NaN,NaN,NaN
NUM|[bitch]+NOUN,0.053504,0.222222,0.248452,NaN,NaN
NUM|[bitch]+NOUN|AUX,0.110068,0.362030,0.333333,0.745356,NaN


In [8]:
to_drop = [c for c in reduced_matrix.columns if any(reduced_matrix[c] ==1)]

In [9]:
len(df.columns.values)

900

In [10]:
len(to_drop)

528

In [11]:
df_reduced = df_new.drop(to_drop, axis=1)
df_reduced.shape

(112, 370)

In [12]:
df_reconstructed = df[df_reduced.columns.values]
df_reconstructed = df_reconstructed.drop("labels", axis=1)
# df_reconstructed["id"] = df['id']
# df_reconstructed["sentences"] = df["sentences"]

In [13]:
inputs = df_reconstructed.values
outs = df["labels"].values



selector = SelectKBest(f_classif, k=5)
X_new = selector.fit_transform(inputs, outs)
cols = selector.get_support(indices=True)
selected_patterns = np.take(df_reconstructed.columns.values,[x for x in cols] )
print(selected_patterns)

['[bitch]' '[bitch]|$LOC' '[hoe]|[bitch]' '[get]|[bitch]' '$DATE|[bitch]']


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(16,12))

# Create a custom diverging palette
cmap = sns.diverging_palette(250, 15, s=75, l=40,
                             n=9, center="light", as_cmap=True)

_ = sns.heatmap(matrix, center=0, annot=True, 
                fmt='.2f', square=True, cmap=cmap)

In [115]:
from sklearn.metrics import precision_recall_fscore_support
collector = {}
labels = df['labels']
all_cols = df.columns.values[4:]
for col in all_cols:
    col_selected = df[col]
    fscore = precision_recall_fscore_support(labels, col_selected,  average="binary")[2]
    collector[col] =  fscore

In [116]:
patterns_selected = []

In [117]:
collector = {k: v for k, v in sorted(collector.items(), key=lambda item: item[1])}

In [118]:
starter = list(collector.keys())[-1]


In [119]:
corr = df.corr()

In [120]:
patterns_selected.append(starter)
to_drop = [c for c in corr.columns if corr[starter][c] >= 0.8]
df_reduced = df.drop(to_drop, axis=1)

In [121]:
import numpy as np
df_new = pd.DataFrame()
new_cols = df_reduced.columns.values[4:]
for coll in new_cols:
    
    df_new[coll] = np.logical_or(df_reduced[coll], df[starter])

/tmp/ipykernel_26584/1975192939.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_new[coll] = np.logical_or(df_reduced[coll], df[starter])


In [122]:
df_new.shape

(18, 101)

In [123]:
new_collector = {}
for col in new_cols:
    col_selected = df_reduced[col]
    fscore = precision_recall_fscore_support(labels, col_selected,  average="binary")[2]
    new_collector[col] =  fscore

In [124]:
starter, patterns_selected

('$PERCENT|[pay]', ['$PERCENT|[pay]'])

In [128]:
new_collector = {k: v for k, v in sorted(new_collector.items(), key=lambda item: item[1])}
new_collector
next_starter = list(new_collector.keys())[-1]
patterns_selected.append(next_starter)

In [127]:
next_starter

'[expensive]|[pay]+*+PRON'

In [129]:
corr = df_reduced.corr()
to_drop = [c for c in corr.columns if corr[next_starter][c] >= 0.8]
df_reduced = df_reduced.drop(to_drop, axis=1)

In [131]:
to_drop

['[pay]|[expensive]',
 '[pay]|[expensive]+*+PRON',
 '[pay]|[expensive]+*+PRON+AUX',
 '[expensive]|[pay]',
 '[expensive]|[pay]+*+PRON',
 '[expensive]|[pay]+*+PRON+AUX']

In [1]:
import torch

def train_and_report(patterns, inputs, outputs):
    #Change numpy inputs to tensors 
    outputs = torch.tensor(outputs).reshape(-1,1)
    inputs = torch.tensor(inputs)

    #train the linear layer for 100 iterations
    #100 chosen at random TODO see what a good number is for iteration

    net = torch.nn.Linear(inputs.shape[1],1, bias=False)
    sigmoid = torch.nn.Sigmoid()

    criterion = torch.nn.BCELoss()
    optimizer = torch.optim.SGD(net.parameters(), lr=0.1)

    losses = []
    net.train()
    for e in range(100):
        optimizer.zero_grad()
        o =  sigmoid.forward(net.forward(inputs.float()))
            
        loss = criterion(o, outputs.float())
            
        losses.append(loss.sum().item())
        loss.backward()
            
        optimizer.step()
    
    pred =  sigmoid.forward(net.forward(inputs.float())).detach().numpy()>0.5
    labeled_prf = precision_recall_fscore_support(outputs, pred, average="weighted")

    fscore = labeled_prf[2]
    # print(f"{patterns}, {fscore}")


    return fscore

/home/simret/miniconda3/envs/py39/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
#imports
import numpy as np
from sklearn.metrics import precision_recall_fscore_support
import pandas as pd

# define df, columns, true labels
def feature_selector(df):

    print(f"==================================Start of Feature Selection===========================================")
    labels = df['labels']
    jj = 0
    ### Controller variables
    patterns_selected = []
    highest_fscore = "0.0"
    df_subset = pd.DataFrame()
    remaining_cols = df.columns.values[4:]



    outputs = df["labels"].values
    while len(patterns_selected)<10 and len(remaining_cols)>0:
        jj += 1
        print(f"Starting iteration {jj} {len(remaining_cols)}")
        #first calculate the fscore
        collector = {}
        local_max_fscore = "0.0"
        for col in remaining_cols:
            col_selected = df[col].astype('int64')
            current_patterns = patterns_selected+[col]
            current_df = pd.concat([df_subset, col_selected], axis=1)
            inputs = current_df.values
            
            # fscore = precision_recall_fscore_support(labels, col_selected,  average="binary")[2]
            fscore = train_and_report(current_patterns, inputs, outputs)
            
                
            exists = str(fscore) in collector
            if(exists):
                collector[str(fscore)].append(col)
                
            else:
                collector[str(fscore)] = [col]
        #sort and get a pattern with high fscore
        selected_starter_pattern = list(collector.values())[-1]
        collector = {k: v for k, v in sorted(collector.items(), key=lambda item: item[0])}
        current_fscore = list(collector.keys())[-1]

        if(current_fscore>highest_fscore):
            highest_fscore = current_fscore
        else:
            # print(f"{highest_fscore} {current_fscore}, {selected_starter_pattern}")
            break
        selected_starter_pattern = list(collector.values())[-1]
        # print(selected_starter_pattern)

        #Group the correlated ones and pick the shortest
        rowss = df[selected_starter_pattern]
        correlation = rowss.corr()
        correlation.loc[:,:] =  np.tril(correlation, k=-1)
        cor = correlation.stack()
        ones = cor[cor >=0.8].reset_index().loc[:,['level_0','level_1']]
        ones = ones.query('level_0 not in level_1')
        grps = list(ones.groupby('level_0').groups.keys())
        colls = []
        for i in grps:
            groups = ones[ones["level_0"]==i].values
            set_maker = []
            for patterns in groups:
                set_maker += patterns.tolist()
            colls.append(sorted(set_maker, key=len)[0])
            
        for selected_starter_pattern in colls:
            patterns_selected.append(selected_starter_pattern)
            df_subset[selected_starter_pattern] = df[selected_starter_pattern].astype('int64')
            try:
                selected_starter_series = df[selected_starter_pattern][0]
                
                corr = df.corr()
                to_drop = [c for c in corr.columns if corr[selected_starter_pattern][c] >= 0.8] #0.8 chosen at random
                df = df.drop(to_drop, axis=1)

                #create a new df with combination of current one
                remaining_cols = df.columns.values[4:]
                for collumn in remaining_cols:
                    df[collumn] = np.logical_or(df[collumn], selected_starter_series)
            except:
                print("We already removed ", selected_starter_pattern)
            for coll in remaining_cols:
                df[coll] = np.logical_or(df[coll], selected_starter_series)
        
        print(f"Finishing iteration {jj} {len(remaining_cols)}, --- {patterns_selected}, {highest_fscore}")
    
    print(f"---------------------------Summary---------------------------")
    print(f"Patterns {patterns_selected}")
    print(f"Positive examples \n{df[df['labels']==1]['sentences'].values}")
    print(f"Negative examples \n{df[df['labels']==0]['sentences'].values}")

    print(f"==================================End of Feature Selection===========================================")
    return patterns_selected




In [27]:
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv("offensive_pats.csv")
print(df.shape)

features = feature_selector(df)

(82, 1371)
==================================Start of Feature Selection===========================================
Starting iteration 1 1367
Finishing iteration 1 1365, --- ['[hoe]|[bitch]'], 0.8790973028777906
Starting iteration 2 1365
---------------------------Summary---------------------------
Patterns ['[hoe]|[bitch]']
Positive examples 
["Die twice bitch &#8220;DivaMonRoe2uHoE: Thank you. &#128536; &#8220;07Cannon: DivaMonRoe2uHoE you're the filthiest hoe of all time "
 'RT Priceless_KP: It ain nun to cut dat bitch awf \n\n- Verizon'
 'lilmenifee__ ohh bitch i gotta go fck with my girl my second lil family frfr'
 '&#8220;Mix_Mommie: I hate when bitches do that "who are you talking to" shit like clearly you.&#8221;I be like "BITCH WIT DA RED DOOR HOE"&#128514;&#128514;'
 'RT PizzaBeforeHoes: Move dat muscle hoe &#128548; \n"Tell me watchu mad foe"&#128170;&#128514; '
 "mel0_yell0w ain't that a bitch."
 "' South Central niggahs really don't be lovin these hoes , these hoes don't be